## **cement strenght prediction model** ##


In [46]:
# import manipulation lybrary
import pandas as pd
import numpy as np

# import visualization lybrary
import matplotlib.pyplot as plt
import seaborn as sns

# import machine learning lybraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder,LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,BaggingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# data ingestion
def data_ingestion():
    df = pd.read_csv(r"C:\Cement_Strenght_Prediction_Model\data\raw\Concrete_Compressive_Strength.csv")
    return df


In [44]:
from collections import OrderedDict
# data exploration 
def data_exploration(df):
    # segregate numerical and categorical columns 
    numerical_col = df.select_dtypes(exclude='object').columns
    categorical_col = df.select_dtypes(include = 'object').columns

    # numerical descriptive stats
    numerical_stats = []

    Q1 = df[numerical_col].quantile(0.25)
    Q3 = df[numerical_col].quantile(0.75)
    IQR = Q3 - Q1
    LW = Q1 - 1.5*IQR
    UW = Q3 + 1.5*IQR
    Outlier_Count = (df[numerical_col] < LW) | (df[numerical_col] > UW)
    Outlier_Percentage = (Outlier_Count.sum()/len(df))*100

    for i in numerical_col:
        num_stats = OrderedDict({
        "Feature":i,
        "Count":df[i].count(),
        "Maximum":df[i].max(),
        "Minimum":df[i].min(),
        "Mean":df[i].mean(),
        "Median":df[i].median(),
        "Q1":Q1,
        "Q3":Q3,
        "IQR":IQR,
        "Lower_Whisker":LW,
        "Upper_Whisker":UW,
        "Outlier_Count": Outlier_Count.sum(),
        "Outlier_Percentage":Outlier_Percentage,
        "Skewness":df[i].skew(),
        "Kurtosis":df[i].kurtosis(),
        "Standard Deviation":df[i].std()

    })
    
        numerical_stats.append(num_stats)
        numerical_stats_report = pd.DataFrame(numerical_stats)
    return numerical_stats_report

numerical_stats_report = data_exploration(df)
numerical_stats_report


,Feature,Count,Maximum,Minimum,Mean,Median,Q1,Q3,IQR,Lower_Whisker,Upper_Whisker,Outlier_Count,Outlier_Percentage,Skewness,Kurtosis,Standard Deviation
0,cement,1030,540.000000,102.000000,281.167864,272.900000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,0.509481,-0.520652,104.506364
1,slag,1030,359.400000,0.000000,73.895825,22.000000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,0.800717,-0.508175,86.279342
2,ash,1030,200.100000,0.000000,54.188350,0.000000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,0.537354,-1.328746,63.997004
3,water,1030,247.000000,121.800000,181.567282,185.000000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,0.074628,0.122082,21.354219
4,superplastic,1030,32.200000,0.000000,6.204660,6.400000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,0.907203,1.411269,5.973841
5,coarseagg,1030,1145.000000,801.000000,972.918932,968.000000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,-0.040220,-0.599016,77.753954
6,fineagg,1030,992.600000,594.000000,773.580485,779.500000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,-0.253010,-0.102177,80.175980
7,age,1030,365.000000,1.000000,45.662136,28.000000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,3.269177,12.168989,63.169912
8,strength,1030,82.600000,2.330000,35.817961,34.445000,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,0.416977,-0.313725,16.705742
9,water_cement_ratio,1030,1.882334,0.266892,0.748263,0.675346,cement 192.375 slag ...,cement 350.0 slag ...,cement 157.625 slag ...,cement -44.0625 slag ...,cement 586.4375 slag ...,cement 0 slag ...,cement 0.0 slag ...,0.958065,0.734109,0.314003


In [23]:
# Step3: Dataset Information
def dataset_info(df):
  print(df.info())

In [24]:
df = data_ingestion()
numerical_stats_report = data_exploration(df)
info = dataset_info(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cement                    1030 non-null   float64
 1   slag                      1030 non-null   float64
 2   ash                       1030 non-null   float64
 3   water                     1030 non-null   float64
 4   superplastic              1030 non-null   float64
 5   coarseagg                 1030 non-null   float64
 6   fineagg                   1030 non-null   float64
 7   age                       1030 non-null   int64  
 8   strength                  1030 non-null   float64
 9   water_cement_ratio        1030 non-null   float64
 10  total_binder              1030 non-null   float64
 11  aggregate_to_cement       1030 non-null   float64
 12  cement_water_interaction  1030 non-null   float64
 13  age_strength_proxy        1030 non-null   float64
dtypes: float

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cement                    1030 non-null   float64
 1   slag                      1030 non-null   float64
 2   ash                       1030 non-null   float64
 3   water                     1030 non-null   float64
 4   superplastic              1030 non-null   float64
 5   coarseagg                 1030 non-null   float64
 6   fineagg                   1030 non-null   float64
 7   age                       1030 non-null   int64  
 8   strength                  1030 non-null   float64
 9   water_cement_ratio        1030 non-null   float64
 10  total_binder              1030 non-null   float64
 11  aggregate_to_cement       1030 non-null   float64
 12  cement_water_interaction  1030 non-null   float64
 13  age_strength_proxy        1030 non-null   float64
dtypes: float

In [27]:
df['age'] = df['age'].astype('Float64')

In [69]:
def preprocessing(df):

    X = df.drop(columns= 'strength')
    y = df['strength']
   
    # use train and test split

    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                        random_state=1,
                                                        test_size=0.3)
    
    # Use Scaling Techniques
    rs = RobustScaler().fit(X_train,y_train)
    X_train= rs.fit_transform(X_train)
    X_test= rs.transform(X_test)

    return X_train, X_test, y_train, y_test
    
X_train, X_test, y_train, y_test = preprocessing(df)
X_train.shape

      



(721, 13)

In [70]:
def model_building(df):
    
    models = {
        "Linear Regression": LinearRegression(),
        "Decision Tree": DecisionTreeRegressor(),
        "Random Forest": RandomForestRegressor(),
        "Gradient Boosting": GradientBoostingRegressor(),
        "Bagging Regressor": BaggingRegressor(),
        "AdaBoost Regressor": AdaBoostRegressor(),
        "Support Vector Regressor": SVR(),
        "K-Neighbors Regressor": KNeighborsRegressor()
        }
    
    return models

models = model_building(df)

In [71]:
from sklearn.metrics import r2_score

def model_evaluation(models, X_train, X_test, y_train, y_test):
    r2scores = {}

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2scores[model_name] = r2_score(y_test, y_pred)

        print(f"Model: {model_name}")
        print(f"R2 Score: {r2scores[model_name]}")

    return r2scores


r2scores = model_evaluation(models, X_train, X_test, y_train, y_test)
print(r2scores)

Model: Linear Regression
R2 Score: 0.8173690536956675
Model: Decision Tree
R2 Score: 0.8513210213992768
Model: Random Forest
R2 Score: 0.9103307535437172
Model: Gradient Boosting
R2 Score: 0.9091899832619942
Model: Bagging Regressor
R2 Score: 0.8963524534709786
Model: AdaBoost Regressor
R2 Score: 0.8069695608322043
Model: Support Vector Regressor
R2 Score: 0.7671012945149633
Model: K-Neighbors Regressor
R2 Score: 0.8077377472879232
{'Linear Regression': 0.8173690536956675, 'Decision Tree': 0.8513210213992768, 'Random Forest': 0.9103307535437172, 'Gradient Boosting': 0.9091899832619942, 'Bagging Regressor': 0.8963524534709786, 'AdaBoost Regressor': 0.8069695608322043, 'Support Vector Regressor': 0.7671012945149633, 'K-Neighbors Regressor': 0.8077377472879232}
